In [1]:
from copy import deepcopy

In [2]:
def nussinov_algor(seq, MinLoop) :
    seq_size = len(seq)
    arr = [[0 for i in range(seq_size)] for j in range(seq_size)]
    paired = [('A','U'),('U','A'),('G','C'),('C','G')]
    for j in range(1,seq_size) :
        for i in range(j-1,-1,-1) :
            max_val = 0
            if j-i >= MinLoop :
                max_val = max(max_val,arr[i+1][j]) #i unpaired
                max_val = max(max_val,arr[i][j-1]) #j unparied
                if (seq[i],seq[j]) in paired :
                    max_val = max(max_val, arr[i+1][j-1] + 1)
                for k in range(i+1,j) :
                    max_val = max(max_val,arr[i][k] + arr[k+1][j])
            arr[i][j] = max_val
    return arr

In [3]:
def traceback(arr, seq, i, j) :
    seq_size = len(seq)
    paired = [('A','U'),('U','A'),('G','C'),('C','G')]
    if (seq[i],seq[j]) in paired and arr[i+1][j-1]+1 == arr[i][j] :
        return '('+ traceback(arr,seq,i+1,j-1) + ')'  #prioritize stem
    if  i > j :
        return ''
    if arr[i][j] == arr[i+1][j] :
        return '.' + traceback(arr, seq, i+1, j)
    if arr[i][j] == arr[i][j-1] :
        return traceback(arr, seq, i, j-1) + '.'
    for k in range(i+1, j) :
        if arr[i][k] + arr[k+1][j] == arr[i][j] :
            return traceback(arr,seq,i,k) + traceback(arr,seq,k+1,j)
    
def nussinov(seq, MinLoop = 4) :
    i = 0
    j = len(seq)-1
    arr = nussinov_algor(seq,MinLoop)
    return traceback(arr,seq,i,j)

In [10]:
seq = "_NNNUUGGUGGCGAUAGCGAAGAGGUCACACCCGUUCCCAUACCGAACACGGAAGUUAAGCUCUUCAGCGCCGAUGGUAGUCGGGGGUUUCCCCCUGUGAGAGUAGGACGCCGCCAAGC"
#print(len(seq))

In [11]:
def calculateConsecPairs(rna,MinStem,MinLoop):
    judge = [('A','U'),('U','A'),('G','C'),('C','G'),('G','U'),('U','G')]
    n = len(rna)-1
    pairs = []
    for i in range(1, n-2*MinStem-MinLoop+1):
        for j in range(i+2*MinStem+MinLoop-1, n+1):
            conPair = 0
            k = 0
            while k <= ((j-i-MinLoop+1)/2):
                if (rna[i+k],rna[j-k]) in judge:
                    conPair += 1
                    if conPair >= MinStem:
                        tempPair = (i,j,conPair)
                        pairs.append(tempPair)
                else:
                    break
                k += 1
    pairs.sort(key = lambda x : -(x[1]-x[0]))
    return pairs


In [23]:
calculateConsecPairs(seq,3,3)

[(4, 116, 3),
 (4, 116, 4),
 (4, 116, 5),
 (4, 116, 6),
 (4, 116, 7),
 (4, 116, 8),
 (4, 116, 9),
 (5, 115, 3),
 (5, 115, 4),
 (5, 115, 5),
 (5, 115, 6),
 (5, 115, 7),
 (5, 115, 8),
 (6, 114, 3),
 (6, 114, 4),
 (6, 114, 5),
 (6, 114, 6),
 (6, 114, 7),
 (5, 112, 3),
 (5, 112, 4),
 (5, 112, 5),
 (8, 115, 3),
 (8, 115, 4),
 (8, 115, 5),
 (7, 113, 3),
 (7, 113, 4),
 (7, 113, 5),
 (7, 113, 6),
 (6, 111, 3),
 (6, 111, 4),
 (9, 114, 3),
 (9, 114, 4),
 (8, 112, 3),
 (8, 112, 4),
 (8, 112, 5),
 (7, 110, 3),
 (10, 113, 3),
 (9, 111, 3),
 (9, 111, 4),
 (4, 105, 3),
 (10, 110, 3),
 (16, 113, 3),
 (4, 99, 3),
 (16, 110, 3),
 (5, 96, 3),
 (23, 114, 3),
 (7, 97, 3),
 (7, 97, 4),
 (8, 96, 3),
 (23, 111, 3),
 (10, 97, 3),
 (33, 118, 3),
 (13, 97, 3),
 (13, 97, 4),
 (24, 108, 3),
 (6, 88, 3),
 (14, 96, 3),
 (16, 97, 3),
 (4, 83, 3),
 (4, 83, 4),
 (4, 83, 5),
 (9, 88, 3),
 (5, 82, 3),
 (5, 82, 4),
 (32, 109, 3),
 (32, 109, 4),
 (32, 109, 5),
 (12, 88, 3),
 (4, 79, 3),
 (6, 81, 3),
 (33, 108, 3),
 (33, 10

In [6]:
class State :
    def __init__(self,seq) :
        self.totalStem = 0 
        self.nodes = []
        self.seqLen = len(seq)
        self.currentState = 0
        
    def insert(self, node) :
        self.nodes.append(node)
        self.totalStem += node.len
        
    def __lt__(self,other) :
        return self.totalStem < other.totalStem
    
    def __eq__(self,other) :
        return self.totalStem == other.totalStem
    
    def __gt__(self,other) :
        return self.totalStem > other.totalStem
    
    def getAnswer(self) :
        return [(node.start,node.end,node.len) for node in self.nodes]
        
        
class Node :
    def __init__(self, start, end, length) :
        self.start = start
        self.end = end
        self.len = length
        
    def __str__(self) :
        return f'{self.start}, {self.end}, {self.len}'
        
class PriorityQueue(object): 
    def __init__(self): 
        self.queue = [] 

    # for checking if the queue is empty 
    def isEmpty(self): 
        return len(self.queue) == 0
  
    # for inserting an element in the queue 
    def push(self, data): 
        self.queue.append(data) 
  
    # for popping an element based on Priority 
    def pop(self): 
        try: 
            currentMax = 0
            for i in range(len(self.queue)): 
                if self.queue[i] > self.queue[currentMax]: 
                    currentMax = i 
            item = self.queue[currentMax] 
            del self.queue[currentMax] 
            return item 
        
        except IndexError: 
            print() 
            exit() 
        
def checkConfilct(state, selectedNode) :
    frontS = selectedNode.start
    backS = selectedNode.end
    
    for node in state.nodes :
        front = node.start+node.len
        back = node.end- node.len
        if (frontS > front and backS < back) or (frontS > back+node.len) or (backS < front-node.len) :
            continue
        else:
            return False        
        
    return True

In [20]:
def getPair(seq) :  
    k = 20
    pairList = []

    while True:
        pairList = calculateConsecPairs(seq, k, 3)
        if len(pairList)!=0:
            pairList = calculateConsecPairs(seq,k-6,3) #editted
            break
        k-=1
    ret = []
    for (start, end, length) in pairList :
        ret += [Node(start,end,length)]
    
    return ret

In [18]:
def run(seq) :
    state = State(seq)
    pq = PriorityQueue()
    pairList = getPair(seq)
    
    state1 = State(seq)   
    state1.insert(pairList[0])
    state1.currentState+=1
    
    state2 = State(seq)
    state2.currentState += 1
    
    pq.push(state1)
    pq.push(state2)
    
    while(not pq.isEmpty()) :
        state = pq.pop()
        cur = state.currentState
        
        print(f'current = {cur}, state = {state.getAnswer()}, total = {state.totalStem}')
        
        #print(len(pairList))
        print(cur)
        
        if cur == len(pairList)-1 :
            return state
        
        cur += 1
        
        state1 = deepcopy(state)
        state1.currentState = cur
        pq.push(state1)
        
        state2 = deepcopy(state)
        if (checkConfilct(state2,pairList[cur])):
            state2.currentState = cur
            state2.insert(pairList[cur])
            pq.push(state2)
      

In [21]:
ans = run(seq)
ans.getAnswer()

current = 1, state = [(4, 116, 3)], total = 3
1
current = 2, state = [(4, 116, 3)], total = 3
2
current = 3, state = [(4, 116, 3)], total = 3
3
current = 4, state = [(4, 116, 3)], total = 3
4
current = 5, state = [(4, 116, 3)], total = 3
5
current = 6, state = [(4, 116, 3)], total = 3
6
current = 7, state = [(4, 116, 3)], total = 3
7
current = 8, state = [(4, 116, 3)], total = 3
8
current = 9, state = [(4, 116, 3)], total = 3
9
current = 10, state = [(4, 116, 3)], total = 3
10
current = 11, state = [(4, 116, 3)], total = 3
11
current = 12, state = [(4, 116, 3)], total = 3
12
current = 13, state = [(4, 116, 3)], total = 3
13
current = 14, state = [(4, 116, 3)], total = 3
14
current = 15, state = [(4, 116, 3)], total = 3
15
current = 16, state = [(4, 116, 3)], total = 3
16
current = 17, state = [(4, 116, 3)], total = 3
17
current = 18, state = [(4, 116, 3)], total = 3
18
current = 19, state = [(4, 116, 3)], total = 3
19
current = 20, state = [(4, 116, 3)], total = 3
20
current = 21, stat

[(4, 116, 3),
 (8, 112, 3),
 (13, 97, 3),
 (18, 90, 3),
 (23, 81, 3),
 (29, 77, 3),
 (33, 68, 3),
 (50, 63, 3)]